<img src="https://s3-ap-southeast-1.amazonaws.com/he-public-data/wordmark_black65ee464.png" width="700">

# Week 2 : Final Challenge

**Welcome to the final challenge!**  

In the previous notebook we've seen how we can use the VQC class in Aqua to classify the digits `0` and `1`. However, classifying `0` and `1` is relatively simple as digits `0` and `1` are easily distinguishable. `4` and `9` however, are notoriously similar, with a _loop_ on the top and a _line_ on the bottom. This can be corroborated looking at our 2-D t-SNE plot from the previous notebook (Fig.2), we see that `0` and `1` are clustered relatively far from each other making them easily distinguishable, however `4` and `9` are overlapping. In this challenge we are providing you with a dataset with digits reduced to **dimension 3**. For example, in Fig.1 we can see the dimension reduction of the 784 dimension vector for digit `4` into a dimension 3 feature vector. 

**Fig.1 : Features of the digit `4` after reducing dimension to 3:** 
<img src="https://s3-ap-southeast-1.amazonaws.com/he-public-data/four2a7701f.png" width="700">

**Fig.2 : MNIST dataset after dimension reduction to 2 as given in the previous notebook:**
<img src="https://s3-ap-southeast-1.amazonaws.com/he-public-data/mnist_plot53adb39.png" width="400">

## Challenge Question   
Use the VQC method from Aqua to classify the digits `4` and `9` as given in the dataset **challenge_dataset_4_9_dim3_training.csv** provided to you. 

## Rules and Guidelines

* Your `QuantumCircuit` can have a **maximum of 6 qubits**.
* **Cost of the circuit should be less than 2000**.  
* You should not change names of the functions `feature_map()` , `variational_circuit()`  and `return_optimal_params()`.
* All the functions must return the value types as mentioned. 
* All circuits must be Qiskit generated.
* Best of all submissions is considered for grading.

## Judging criteria 

* Primary judgement is based on the **accuracy of the model**, higher the better. **Accuracies which differ by less than 0.005 will be considered to be equal**. ex: Accuracies 0.7783 and 0.7741 will be considered to be equal.
* If the accuracies are tied, the tie will be broken using **cost of the circuit** as the metric, lower the better. 
* In the case that both accuracy of the model and cost of the circuit are equal, **time of submission** is taken into account, Earlier the better. 

_**Important Note:**_ The **leaderboard shown during the progress of the competition** will only display accuracy of the model and is **not the final leaderboard**. Breaking ties between accuracy of the model by considering lower **cost of circuit** will only be done after the competition ends. **The final leaderboard will be announced post the event** which will take into consideration cost of the circuit and time of submission. 

## Certificate Eligibility

Everyone who scores an **accuracy greater than 0.70 (i.e, 70%) will be eligible for a certificate**. 


An explanation on how to calculate the accuracy of the model and the cost of the circuit is given in the end inside the `grade()` function. Before you submit, make sure the grading function is running on your device. To save time you can also use the grading function provided to calculate the accuracy and circuit cost without having to submit your solution onto HackerEarth. Remember, your final score will be determined using the same grading methods as given in this notebook, but will be evaluated on unseen datapoints.

In [1]:
# installing a few dependencies
!pip install --upgrade seaborn==0.10.1
!pip install --upgrade scikit-learn==0.23.1
!pip install --upgrade matplotlib==3.2.0
!pip install --upgrade pandas==1.0.4
!pip install --upgrade qiskit==0.19.6 
!pip install --upgrade plotly==4.9.0

# the output will be cleared after installation
from IPython.display import clear_output
clear_output()

In [68]:
# we have imported a few libraries we thing might be useful 
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit import Aer

from qiskit import *
import numpy as np
from qiskit.visualization import plot_bloch_multivector, plot_histogram
%matplotlib inline
import matplotlib.pyplot as plt

import time
from qiskit.circuit.library import ZZFeatureMap, ZFeatureMap, PauliFeatureMap, RealAmplitudes, EfficientSU2
from qiskit.aqua.utils import split_dataset_to_data_and_labels, map_label_to_class_name
from qiskit.aqua import QuantumInstance
from qiskit.aqua.algorithms import VQC
from qiskit.aqua.components.optimizers import COBYLA,ADAM, SPSA, SLSQP


# The the write_and_run() magic function creates a file with the content inside the cell that it is run. 
# You have used this in previous exercises for creating your submission files. 
# It will be used for the same purpose here.

from IPython.core.magic import register_cell_magic
@register_cell_magic
def write_and_run(line, cell):
    argz = line.split()
    file = argz[-1]
    mode = 'w'
    with open(file, mode) as f:
        f.write(cell)
    get_ipython().run_cell(cell)

# Solution

## Data loading 

This notebook has helper functions and code snippets to save your time and help you concentrate on what's important: Increasing the accuracy of your model. Running the cell below will import the challenge dataset and will be available to you as `data`. Before running the cell below store the dataset in this file structure (or change the `data_path` accordingly):  

- `challenge_notebook.ipynb`
- `dataset`
    - `challenge_dataset_4_9.csv`


In [36]:
data_path=''
data = np.loadtxt(data_path + "challenge_dataset_4_9_dim3_training.csv", delimiter=",")

# extracting the first column which contains the labels
data_labels = data[:, :1].reshape(data.shape[0],)
# extracting all the columns but the first which are our features
data_features = data[:, 1:]

## Visualizing the dataset

Before we dive into solving the question it is always beneficial to look at the dataset pictographically. This will help us understand patterns which we could leverage when designing our feature maps and variational circuits for example.

In [37]:
import plotly.express as px
import pandas as pd

# creating a dataframe using pandas only for the purpose fo plotting
df = pd.DataFrame({'Component 0':data_features[:,0], 'Component 1':data_features[:,1], 
                   'Component 2':data_features[:,2], 'label':data_labels})

fig = px.scatter_3d(df, x='Component 0', y='Component 1', z='Component 2', color='label')
fig.show()

## Extracting the training dataset

The given dataset has already been reduced in dimension and normalized, so, further pre-processing isn't techincally required. You can do so if you want to, but the testing dataset will be of the same dimension and normalisation as the training dataset provided. Training a dataset of size 6,000 will take multiple hours so you'll need to extract a subset of the dataset to use as a training dataset. The accuracy of the model may vary based on the datapoints and size of the training dataset you choose. Thus, experimenting with various sizes and datapoints will be necessary. For example, Increasing the training dataset size may increase the accuracy of the model however it will increase the training time as well.

Use the space below to extract your training dataset from `data`. For your convenience `data` has been segregated into `data_labels` and `data_features`.

* `data_labels` : 6,000 $\times$ 1 column vector with each entry either `4` or `9` 
* `data_features` : 6,000 $\times$ 3 matrix with each row having the feature corresponding to the label in `data_labels`

**Note:** This process was done in the previous [VQC notebook](https://github.com/Qiskit-Challenge-India/2020/blob/master/Day%206%2C%207%2C8/VQC_notebook.ipynb) with `0` and `1` labels and can be modified and used here as well. 

In [152]:
### WRITE YOUR CODE BETWEEN THESE LINES - START

# do your classical pre-processing here

four_datapoints = []
nine_datapoints = []

for i in range(6000):
    if data_labels[i] == 4:                   # extracting 4s
        four_datapoints.append(data_features[i])

for i in range(6000):
    if data_labels[i] == 9:                   # extracting 9s
        nine_datapoints.append(data_features[i])
        
four_datapoints = np.array(four_datapoints)
nine_datapoints = np.array(nine_datapoints)

train_size = 20
test_size = 10

four_train = four_datapoints[:train_size]
nine_train = nine_datapoints[:train_size]

four_test = four_datapoints[train_size + 1:train_size + test_size + 1]
nine_test = nine_datapoints[train_size + 1:train_size + test_size + 1]

# store your training and testing datasets to be input in the VQC optimizer in the "training_input" and 
# "testing_input" variables respectively. These variables will be accessed while creating a VQC instance later. 
training_input = {'A':four_train, 'B':nine_train}
test_input = {'A':four_test, 'B':nine_test}

### WRITE YOUR CODE BETWEEN THESE LINES - END

## Building a Quantum Feature Map

Given below is the `feature_map()` function. It takes no input and has to return a feature map which is either a `FeatureMap` or `QuantumCircuit` object. In the previous notebook you've learnt how feature maps work and the process of using existing feature maps in Qiskit or creating your own. In the space given **inside the function** you have to create a feature map and return it.   


**IMPORTANT:** 
* If you require Qiskit import statements other than the ones provided in the cell below, please include them inside the appropriate space provided. **All additional import statements must be Qiskit imports.** 
* the first line of the cell below must be `%%write_and_run feature_map.py`. This function stores the content of the cell below in the file `feature_map.py`

In [153]:
%%write_and_run feature_map.py
# the write_and_run function writes the content in this cell into the file "feature_map.py"

### WRITE YOUR CODE BETWEEN THESE LINES - START
    
# import libraries that are used in the function below.
from qiskit import QuantumCircuit
from qiskit.circuit import ParameterVector
from qiskit.circuit.library import ZZFeatureMap, ZFeatureMap, PauliFeatureMap
    
### WRITE YOUR CODE BETWEEN THESE LINES - END

def feature_map(): 
    # BUILD FEATURE MAP HERE - START
    
    # import required qiskit libraries if additional libraries are required
    
    # build the feature map
    # feature_map = ZZFeatureMap(feature_dimension=3, reps=3, entanglement='full')
    feature_map = PauliFeatureMap(feature_dimension=3, reps=3, paulis=['Z','ZZ','XY'])
    # BUILD FEATURE MAP HERE - END
    
    #return the feature map which is either a FeatureMap or QuantumCircuit object
    return feature_map

## Building a Variational Circuit

Given below is the `variational_circuit()` function. It takes no input and has to return a variational circuit which is either a `VariationalForm` or `QuantumCircuit` object. In the previous notebook you've learnt how variational circuits work and the process of using existing variational circuit in Qiskit or creating your own. You have to create a variational circuit in the space given **inside the function** and return it. You can find various variational circuits in the [Qiskit Circuit Library](https://qiskit.org/documentation/apidoc/circuit_library.html) under N-local circuits.

**IMPORTANT:** 
* If you require Qiskit import statements other than the ones provided in the cell below, please include them inside the appropriate space provided. **All additional import statements must be Qiskit imports.** 
* the first line of the cell below must be `%%write_and_run feature_map.py`. This function stores the content of the cell below in the file `variational_circuit.py`

In [154]:
%%write_and_run variational_circuit.py
# the write_and_run function writes the content in this cell into the file "variational_circuit.py"

### WRITE YOUR CODE BETWEEN THESE LINES - START
    
# import libraries that are used in the function below.
from qiskit import QuantumCircuit
from qiskit.circuit import ParameterVector
from qiskit.circuit.library import  RealAmplitudes, EfficientSU2
    
### WRITE YOUR CODE BETWEEN THESE LINES - END

def variational_circuit():
    # BUILD VARIATIONAL CIRCUIT HERE - START
    
    # import required qiskit libraries if additional libraries are required
    
    # build the variational circuit
    var_circuit = EfficientSU2(3, reps=2)

    # BUILD VARIATIONAL CIRCUIT HERE - END
    
    # return the variational circuit which is either a VaritionalForm or QuantumCircuit object
    return var_circuit

## Choosing a Classical Optimizer

In the `classical_optimizer()` function given below you will have to import the optimizer of your choice from [`qiskit.aqua.optimizers`](https://qiskit.org/documentation/apidoc/qiskit.aqua.components.optimizers.html) and return it. This function will not be called by the grading function `grade()` and thus the name of the function `classical_optimizer()`can be changed if needed. 

In [155]:
def classical_optimizer():
    # CHOOSE AND RETURN CLASSICAL OPTIMIZER OBJECT - START
    
    # import the required clasical optimizer from qiskit.aqua.optimizers
    
    # create an optimizer object
    cls_opt = COBYLA(maxiter=500, tol=0.001)

    
    # CHOOSE AND RETURN CLASSICAL OPTIMIZER OBJECT - END
    return cls_opt

### Callback Function

The `VQC` class can take in a callback function to which the following parameters will be passed after every optimization cycle of the algorithm:

* `eval_count` : the evaulation counter
* `var_params` : value of parameters of the variational circuit
* `eval_val`  : current cross entropy cost 
* `index` : the batch index

In [156]:
def call_back_vqc(eval_count, var_params, eval_val, index):
    print("eval_count: {}".format(eval_count))
    print("var_params: {}".format(var_params))
    print("eval_val: {}".format(eval_val))
    print("index: {}".format(index))

## Optimization Step

This is where the whole VQC algorithm will come together. First we create an instance of the `VQC` class. 

In [157]:
# a fixed seed so that we get the same answer when the same input is given. 
seed = 10598

# setting our backend to qasm_simulator with the "statevector" method on. This particular setup is given as it was 
# found to perform better than most. Feel free to play around with different backend options.
backend = Aer.get_backend('qasm_simulator')
backend_options = {"method": "statevector"}

# creating a quantum instance using the backend and backend options taken before
quantum_instance = QuantumInstance(backend, shots=1024, seed_simulator=seed, seed_transpiler=seed, 
                                   backend_options=backend_options)

# creating a VQC instance which you will be used for training. Make sure you input the correct training_dataset and 
# testing_dataset as defined in your program.
vqc = VQC(optimizer=classical_optimizer(), 
          feature_map=feature_map(), 
          var_form=variational_circuit(), 
          callback=call_back_vqc, 
          training_dataset=training_input,     # training_input must be initialized with your training dataset
          test_dataset=test_input)             # testing_input must be initialized with your testing dataset

Now, let's run the VQC classification routine

In [158]:
start = time.process_time()

result = vqc.run(quantum_instance)

print("time taken: ")
print(time.process_time() - start)

print("testing success ratio: {}".format(result['testing_accuracy']))

eval_count: 0
var_params: [-0.62600089 -0.61642749  0.30931466 -0.51022107 -1.40144975 -0.53158874
  0.06397512 -0.48675049 -0.20624046  0.68239383 -0.8653374  -0.33195362
  0.68933117 -0.32941799  0.36805848 -0.09788664 -1.43941358  0.15612631]
eval_val: 0.8137882877038489
index: 0
eval_count: 1
var_params: [ 0.37399911 -0.61642749  0.30931466 -0.51022107 -1.40144975 -0.53158874
  0.06397512 -0.48675049 -0.20624046  0.68239383 -0.8653374  -0.33195362
  0.68933117 -0.32941799  0.36805848 -0.09788664 -1.43941358  0.15612631]
eval_val: 0.8174053729303626
index: 1
eval_count: 2
var_params: [-0.62600089  0.38357251  0.30931466 -0.51022107 -1.40144975 -0.53158874
  0.06397512 -0.48675049 -0.20624046  0.68239383 -0.8653374  -0.33195362
  0.68933117 -0.32941799  0.36805848 -0.09788664 -1.43941358  0.15612631]
eval_val: 0.9085474926747181
index: 2
eval_count: 3
var_params: [-0.62600089 -0.61642749  1.30931466 -0.51022107 -1.40144975 -0.53158874
  0.06397512 -0.48675049 -0.20624046  0.68239383 

eval_count: 29
var_params: [-0.15581224 -1.73970298  2.38514815  0.86795308 -0.80232934  0.46366185
  0.06397512 -0.48675049 -0.20624046  1.70683966 -0.56517527  0.67476995
  1.79792073 -1.11268301  0.80008724  1.07850131 -1.79673323 -0.6610728 ]
eval_val: 0.48042130172047637
index: 29
eval_count: 30
var_params: [-0.15581224 -1.73970298  2.38514815  0.86795308 -0.80232934  0.46366185
  0.06397512  0.01324951 -0.20624046  1.70683966 -0.56517527  0.67476995
  1.79792073 -1.11268301  0.80008724  1.07850131 -1.79673323 -0.6610728 ]
eval_val: 0.48042130172047637
index: 30
eval_count: 31
var_params: [ 0.00945429 -1.96944866  2.68154244  1.0556409  -0.90826537  0.40710188
  0.06397512 -0.48675049 -0.20624046  1.7118092  -0.6462148   0.67613678
  1.81999568 -1.30004964  0.73510694  1.11435886 -1.86937201  0.19299628]
eval_val: 0.5005489054584696
index: 31
eval_count: 32
var_params: [-0.15581224 -1.73970298  2.38514815  0.86795308 -0.80232934  0.46366185
  0.06397512 -0.48675049  0.29375954  1.

eval_count: 58
var_params: [-0.15817358 -1.42848412  2.43425975  0.77822369 -0.75691223  0.7657511
 -0.31629702 -0.44535063 -0.1648406   2.23467726 -1.01371538  1.389589
  1.72482016 -1.28495695  0.84735535  1.06374598 -1.89172978 -0.73773966]
eval_val: 0.45165112560767007
index: 58
eval_count: 59
var_params: [-0.16306747 -1.47936739  2.46571535  0.81916934 -0.68036024  0.74764649
 -0.32293219 -0.21661941 -0.28966277  1.75526754 -1.01757066  1.40537367
  1.74375661 -1.27243339  0.8669823   1.06146047 -1.92619172 -0.76772616]
eval_val: 0.44506695160440424
index: 59
eval_count: 60
var_params: [-0.18833597 -1.44139002  2.50360025  0.84597465 -0.6860237   0.78361378
 -0.31412792 -0.17052015  0.10998988  1.77639989 -1.0670594   1.03735511
  1.72294608 -1.32661222  0.84644127  1.06705011 -1.91548557 -0.82012102]
eval_val: 0.4514898589519817
index: 60
eval_count: 61
var_params: [-0.18121727 -1.36047684  2.54540527  0.78562227 -0.76979653  0.75016573
 -0.36574686 -0.43367875 -0.15316871  1.721

eval_count: 87
var_params: [-0.39316285 -1.41995493  2.41623839  0.84320488 -0.87080769  0.7640001
 -0.4865715  -0.34725857 -0.06674853  2.02836508 -0.8420267   1.70078462
  1.73966729 -1.26201765  0.71045096  1.08283356 -1.55603881 -0.85801671]
eval_val: 0.42975895526392877
index: 87
eval_count: 88
var_params: [-0.32811505 -1.32771091  2.38320336  0.82752277 -0.87690339  0.66619853
 -0.47617905 -0.33842057 -0.05791054  2.0034737  -0.9707278   1.61481801
  1.7044704  -1.22923272  0.72811639  1.21125958 -1.64530377 -0.80781685]
eval_val: 0.43236387475529464
index: 88
eval_count: 89
var_params: [-0.38854473 -1.47294853  2.44866722  0.70495693 -0.72788087  0.6776766
 -0.48763123 -0.32781505 -0.04730501  2.04397367 -0.93301858  1.67419036
  1.78947989 -1.17748005  0.77739794  1.2754936  -1.6548135  -0.7946987 ]
eval_val: 0.43408143927320425
index: 89
eval_count: 90
var_params: [-0.3249204  -1.42338053  2.46264355  0.88530545 -0.86161103  0.6520869
 -0.50157599 -0.32214809 -0.04163805  2.02

eval_count: 116
var_params: [-0.30407343 -1.398376    2.45635036  0.84988472 -0.80691633  0.66500617
 -0.48282686 -0.33924546 -0.05873543  2.04606436 -0.86182436  1.69766028
  1.80368768 -1.16642299  0.87994813  1.22801916 -1.61403503 -0.69326295]
eval_val: 0.42982693275494543
index: 116
eval_count: 117
var_params: [-0.30098174 -1.39159468  2.49352393  0.82532818 -0.78873157  0.68266755
 -0.52676002 -0.33179527 -0.05128524  2.01771833 -0.88527804  1.64830593
  1.7749601  -1.15491731  0.83877056  1.27843756 -1.55104838 -0.73633335]
eval_val: 0.4293123841280595
index: 117
eval_count: 118
var_params: [-0.26492347 -1.43363668  2.47553344  0.83632961 -0.82208542  0.69372774
 -0.53560228 -0.34851675 -0.06800672  1.98941481 -0.864506    1.68352189
  1.82155772 -1.23191305  0.7843557   1.32302898 -1.57644339 -0.70417559]
eval_val: 0.4289204155893467
index: 118
eval_count: 119
var_params: [-0.30957329 -1.40667678  2.4961461   0.84667789 -0.82649508  0.69188911
 -0.51683227 -0.33457138 -0.054061

eval_count: 145
var_params: [-0.34496196 -1.39935619  2.47285276  0.79125577 -0.84502093  0.67380829
 -0.49068387 -0.36923288 -0.08872285  2.02253492 -0.92663857  1.65898315
  1.76281281 -1.17887518  0.8272898   1.31047131 -1.57688897 -0.69630663]
eval_val: 0.42692480059397964
index: 145
eval_count: 146
var_params: [-0.3454516  -1.4138075   2.47512434  0.79066959 -0.84809379  0.68560086
 -0.49509475 -0.36546474 -0.0868447   2.01478751 -0.9198832   1.6607285
  1.76873337 -1.18091309  0.81997954  1.29522159 -1.56143573 -0.70846082]
eval_val: 0.424435754918808
index: 146
eval_count: 147
var_params: [-0.34216934 -1.42410937  2.47303427  0.787789   -0.85736913  0.67756066
 -0.50175074 -0.36832992 -0.07849259  2.02135751 -0.91328132  1.66055137
  1.78124827 -1.17683554  0.80400259  1.29147213 -1.55639783 -0.7040658 ]
eval_val: 0.4251839388750377
index: 147
eval_count: 148
var_params: [-0.34630977 -1.41339656  2.48422535  0.79059898 -0.84460195  0.6859863
 -0.49817295 -0.36474683 -0.08576619 

eval_count: 174
var_params: [-0.35027649 -1.41262086  2.48127598  0.79227742 -0.84969182  0.68332513
 -0.49668088 -0.36744814 -0.08938161  2.01517125 -0.91714383  1.65998311
  1.76665941 -1.1800553   0.82060725  1.29817111 -1.56057905 -0.71496725]
eval_val: 0.42454782461026996
index: 174
eval_count: 175
var_params: [-0.34751842 -1.41223043  2.47920262  0.78827759 -0.85031833  0.68457289
 -0.49851966 -0.36685041 -0.08851617  2.01451126 -0.92129884  1.66145437
  1.76870077 -1.18086593  0.81899681  1.294411   -1.56046582 -0.71436138]
eval_val: 0.42456975241263956
index: 175
eval_count: 176
var_params: [-0.34764598 -1.40977836  2.4773515   0.78790348 -0.85089246  0.68136266
 -0.49656263 -0.3684546  -0.08934282  2.01896832 -0.92257529  1.66098424
  1.76670809 -1.18184297  0.81941877  1.29727536 -1.56528051 -0.71278583]
eval_val: 0.42445754781634504
index: 176
eval_count: 177
var_params: [-0.34913074 -1.40958926  2.4777963   0.78971703 -0.84896346  0.68407056
 -0.49652015 -0.36643857 -0.0879

eval_count: 203
var_params: [-0.34988914 -1.40857603  2.47821271  0.79042187 -0.84846587  0.6840421
 -0.49763973 -0.3685696  -0.0866172   2.01509766 -0.920837    1.66010493
  1.76953409 -1.1831716   0.81756647  1.29503155 -1.56061022 -0.71359231]
eval_val: 0.42399320820624337
index: 203
eval_count: 204
var_params: [-0.3498407  -1.40870924  2.4782152   0.790573   -0.8480618   0.68384556
 -0.49756255 -0.36809007 -0.08693718  2.01511355 -0.92098234  1.65968804
  1.77016234 -1.18327942  0.81777548  1.29492179 -1.56064903 -0.71374877]
eval_val: 0.42400535200611034
index: 204
eval_count: 205
var_params: [-0.35011865 -1.40855314  2.47827022  0.79051477 -0.84838477  0.68389709
 -0.49786694 -0.36832309 -0.08687427  2.01519587 -0.92064163  1.65984411
  1.7694761  -1.18296857  0.81771307  1.29513962 -1.56050526 -0.71343682]
eval_val: 0.42399437482710256
index: 205
eval_count: 206
var_params: [-0.35012531 -1.40879922  2.47840322  0.79073232 -0.84840599  0.68389235
 -0.49746537 -0.36859625 -0.08713

## Storing the optimal parameters for grading

Once the training step of the vqc algorithm is done we obtain the optimal parameters for our specific variational form. For the grading function to be able to access these optimal parameters you will need to follow the steps below. 

* **Step 1**: Run the cell below with `print(repr(vqc.optimal_params))`. 
* **Step 2**: Copy the matrix of optimal parameters and store it in the variable `optimal_parameters` inside the function `return_optimal_params()` in the next cell. This will enable us to extract it while calculating the accuracy your the model during grading. Given below is a pictographical explanation of the same:  

<img src="https://s3-ap-southeast-1.amazonaws.com/he-public-data/opt_params456b075.png" width="800">

In [159]:
print(repr(vqc.optimal_params))

array([-0.34961958, -1.40874408,  2.47835752,  0.79027927, -0.84840324,
        0.68443884, -0.49760568, -0.3681459 , -0.08704838,  2.015284  ,
       -0.92040904,  1.66010337,  1.76969544, -1.18256876,  0.81739621,
        1.29532048, -1.56101432, -0.71368744])


In [161]:
%%write_and_run optimal_params.py
# # the write_and_run function writes the content in this cell into the file "optimal_params.py"

### WRITE YOUR CODE BETWEEN THESE LINES - START
    
# import libraries that are used in the function below.
import numpy as np
    
### WRITE YOUR CODE BETWEEN THESE LINES - END

def return_optimal_params():
    # STORE THE OPTIMAL PARAMETERS AS AN ARRAY IN THE VARIABLE optimal_parameters 
    
    optimal_parameters = [-0.34961958, -1.40874408,  2.47835752,  0.79027927, -0.84840324,
        0.68443884, -0.49760568, -0.3681459 , -0.08704838,  2.015284  ,
       -0.92040904,  1.66010337,  1.76969544, -1.18256876,  0.81739621,
        1.29532048, -1.56101432, -0.71368744]
    
    # STORE THE OPTIMAL PARAMETERS AS AN ARRAY IN THE VARIABLE optimal_parameters 
    return np.array(optimal_parameters)

## Submission

Before we go any further, check that you have the three files `feature_map.py`, `variational_circuit.py` and `optimal_params.py` in the **same working directory as this notebook**. If you do not, then go back to the start and run the notebook making sure you have filled in the code where its required. When you run the cell below, all the three files `feature_map.py`, `variational_circuit.py` and `optimal_params.py` are combined into one file named **"answer.py"**. Now your working directory will have four python (.py) files out of which **"answer.py"** is the submission file: 
* `answer.py` <- upload this file onto HackerEarth and click on "Submit and Evaluate"
* `feature_map.py`
* `variational_circuit.py`
* `optimal_params.py`

In [162]:
solution = ['feature_map.py','variational_circuit.py','optimal_params.py']
file = open("answer.py","w")
file.truncate(0)
for i in solution:    
    with open(i) as f:
        with open("answer.py", "a") as f1:
            for line in f:
                f1.write(line)
file.close()

## Grading Function

Given below is the grading function that we shall use to grade your submission with a test dataset that is of the same format as `challenge_dataset_4_9.csv`. You can use it to grade your submission by extracting a few points out of the `challenge_dataset_4_9.csv` to get a basic idea of how your model is performing. 

In [163]:
#imports required for the grading function 
from qiskit import *
from qiskit.aqua import QuantumInstance
from qiskit.aqua.algorithms import VQC
from qiskit.aqua.components.feature_maps import FeatureMap
from qiskit.aqua.components.variational_forms import VariationalForm
import numpy as np

### Working of the grading function

The grading function `grade()` takes as **input**: 

* `test_data`: (`np.ndarray`) -- **no. of datapoints $\times$ dimension of data** : the datapoints against which we want to test our model. 


* `test_labels`: (`np.ndarray`) -- **no. of datapoints $\times$ 1** : A column vector with each entry either 0 or 1 as entries.


* `feature_map`: (`QuantumCircuit` or `FeatureMap`) -- A quantum feature map which is the output of `feature_map()` defined earlier.


* `variational_form`: (`QuantumCircuit` or `VariationalForm`) -- A variational form which is the output of `variational_circuit()` defined earlier.


* `optimal_params`: (`numpy.ndarray`) -- the optimal parameters obtained after running the VQC algorithm above. These are the values obtained when the function `return_optimal_params()` is run. 


* `find_circuit_cost` : (`bool`) -- Calculates the circuit cost if set to `True`. Circuit cost is calculated by converting the circuit to the basis gate set `\[ 'u3', 'cx'\]` and then applying the formula **cost = 1$\times$(no.of u3 gates) + 10$\times$(no.of cx gates)**.


* `verbose` : (`bool`) -- prints the result message if set to `True`.

And gives as **output**: 

* `model_accuracy` : (`numpy.float64`) -- percent accuracy of the model. 


* `circuit_cost`: (`int`) -- circuit cost as explained above.


* `ans`: (`tuple`) -- Output of the `VQC.predict()` method. 


* `result_msg`: (`str`) -- Result message which also outputs the error message in case of one.


* `unrolled_circuit`: (`QuantumCircuit` or `None`) -- the circuit obtained after unrolling the full VQC circuit and substituting the optimal parameters to the basis gate set `\[ 'u3', 'cx'\]`.

**Note:** if you look inside the `grade()` function in Section 2 you'll see that we have initialized a COBYLA optimizer though the prediction step will not require one. Similarily we have given a dataset to `training dataset`. Both of these are dummy variables. The reason for this is because these are not optional variables the `VQC` class instantiation.  

In [164]:
def grade(test_data, test_labels, feature_map, variational_form, optimal_params, find_circuit_cost=True, verbose=True):
    seed = 10598
    model_accuracy = None 
    circuit_cost=None 
    ans = None
    unrolled_circuit = None
    result_msg=''
    data_dim = np.array(test_data).shape[1]
    dataset_size = np.array(test_data).shape[0]
    dummy_training_dataset=training_input = {'A':np.ones((2,data_dim)), 'B':np.ones((2, data_dim))}
    
    # converting 4's to 0's and 9's to 1's for checking 
    test_labels_transformed = np.where(test_labels==4, 0., 1.)
    max_qubit_count = 6
    max_circuit_cost = 2000
    
    # Section 1
    if feature_map is None:
        result_msg += 'feature_map variable is None. Please submit a valid entry' if verbose else ''
    elif variational_form is None: 
        result_msg += 'variational_form variable is None. Please submit a valid entry' if verbose else ''
    elif optimal_params is None: 
        result_msg += 'optimal_params variable is None. Please submit a valid entry' if verbose else ''
    elif test_data is None: 
        result_msg += 'test_data variable is None. Please submit a valid entry' if verbose else ''
    elif test_labels is None: 
        result_msg += 'test_labels variable is None. Please submit a valid entry' if verbose else ''
    elif not isinstance(feature_map, (QuantumCircuit, FeatureMap)):
        result_msg += 'feature_map variable should be a QuantumCircuit or a FeatureMap not (%s)' % \
                      type(feature_map) if verbose else ''
    elif not isinstance(variational_form, (QuantumCircuit, VariationalForm)):
        result_msg += 'variational_form variable should be a QuantumCircuit or a VariationalForm not (%s)' % \
                      type(variational_form) if verbose else ''
    elif not isinstance(test_data, np.ndarray):
        result_msg += 'test_data variable should be a numpy.ndarray not (%s)' % \
                      type(test_data) if verbose else ''
    elif not isinstance(test_labels, np.ndarray):
        result_msg += 'test_labels variable should be a numpy.ndarray not (%s)' % \
                      type(test_labels) if verbose else ''
    elif not isinstance(optimal_params, np.ndarray):
        result_msg += 'optimal_params variable should be a numpy.ndarray not (%s)' % \
                      type(optimal_params) if verbose else ''
    elif not dataset_size == test_labels_transformed.shape[0]:
        result_msg += 'Dataset size and label array size must be equal'
    # Section 2
    else:
        
        # setting up COBYLA optimizer as a dummy optimizer
        from qiskit.aqua.components.optimizers import COBYLA
        dummy_optimizer = COBYLA()

        # setting up the backend and creating a quantum instance
        backend = Aer.get_backend('qasm_simulator')
        backend_options = {"method": "statevector"}
        quantum_instance = QuantumInstance(backend, 
                                           shots=2000, 
                                           seed_simulator=seed, 
                                           seed_transpiler=seed, 
                                           backend_options=backend_options)

        # creating a VQC instance and running the VQC.predict method to get the accuracy of the model 
        vqc = VQC(optimizer=dummy_optimizer, 
                  feature_map=feature_map, 
                  var_form=variational_form, 
                  training_dataset=dummy_training_dataset)
        
        from qiskit.transpiler import PassManager
        from qiskit.transpiler.passes import Unroller
        pass_ = Unroller(['u3', 'cx'])
        pm = PassManager(pass_)
        # construct circuit with first datapoint
        circuit = vqc.construct_circuit(data[0], optimal_params)
        unrolled_circuit = pm.run(circuit)
        gates = unrolled_circuit.count_ops()
        if 'u3' in gates: 
            circuit_cost = gates['u3']
        if 'cx' in gates: 
            circuit_cost+= 10*gates['cx']
        
        if circuit.num_qubits > max_qubit_count:
            result_msg += 'Your quantum circuit is using more than 6 qubits. Reduce the number of qubits used and try again.'
        elif circuit_cost > max_circuit_cost:
            result_msg += 'The cost of your circuit is exceeding the maximum accpetable cost of 2000. Reduce the circuit cost and try again.'
        else: 
            
            ans = vqc.predict(test_data, quantum_instance=quantum_instance, params=np.array(optimal_params))
            model_accuracy = np.sum(np.equal(test_labels_transformed, ans[1]))/len(ans[1])

            result_msg += 'Accuracy of the model is {}'.format(model_accuracy) if verbose else ''
            result_msg += ' and circuit cost is {}'.format(circuit_cost) if verbose else ''
            
    return model_accuracy, circuit_cost, ans, result_msg, unrolled_circuit

## Process of grading using a dummy grading dataset

Let us create a dummy grading dataset with features and labels `grading_features` and `grading_labels` created from the last 2000 datapoints from `data_features` and `data_labels`so that we can a rough estimate of our accuaracy. It must be noted that this may not be a balanced dataset, i.e, may not have equal number of `4`'s and `9`'s and is not best practice. This is only given for the purpose of the demo of `grade()` function. In the final scoring done on HackerEarth, the testing dataset used will have a balanced number of class labels `4` and `9`.

In [165]:
grading_dataset_size=6000    # this value is not per digit but in total
grading_features = data_features[-grading_dataset_size:]
grading_labels = data_labels[-grading_dataset_size:]

In [166]:
start = time.process_time()

accuracy, circuit_cost, ans, result_msg, full_circuit  =  grade(test_data=grading_features, 
                                                                test_labels=grading_labels, 
                                                                feature_map=feature_map(), 
                                                                variational_form=variational_circuit(), 
                                                                optimal_params=return_optimal_params())

print("time taken: {} seconds".format(time.process_time() - start))
print(result_msg)

time taken: 159.52893072999996 seconds
Accuracy of the model is 0.7556666666666667 and circuit cost is 510


You can also check your **accuracy**, **circuit_cost** and **full_circuit** which is the result of combining the feature map and variational circuit and unrolling into the basis \['u3', 'cx'\].

In [167]:
print("Accuracy of the model: {}".format(accuracy))
print("Circuit Cost: {}".format(circuit_cost))
print("The complete unrolled circuit: ")
full_circuit.draw()

Accuracy of the model: 0.7556666666666667
Circuit Cost: 510
The complete unrolled circuit: 


┌───────────────┐  ┌───────────┐                                     »
q_0: ┤ U3(pi/2,0,pi) ├──┤ U3(0,0,8) ├─────■───────────────────────■────■──»
     ├───────────────┤┌─┴───────────┴──┐┌─┴─┐┌─────────────────┐┌─┴─┐  │  »
q_1: ┤ U3(pi/2,0,pi) ├┤ U3(0,0,1.6456) ├┤ X ├┤ U3(0,0,-3.9809) ├┤ X ├──┼──»
     ├───────────────┤├────────────────┤└───┘└─────────────────┘└───┘┌─┴─┐»
q_2: ┤ U3(pi/2,0,pi) ├┤ U3(0,0,1.8033) ├─────────────────────────────┤ X ├»
     └───────────────┘└────────────────┘                             └───┘»
c_0: ═════════════════════════════════════════════════════════════════════»
                                                                          »
c_1: ═════════════════════════════════════════════════════════════════════»
                                                                          »
c_2: ═════════════════════════════════════════════════════════════════════»
                                                                          »
«                             ┌─────────────────────┐                       »
«q_0: ─────────────────────■──┤ U3(pi/2,-pi/2,pi/2) ├───────────────────────»
«                          │  └─────────────────────┘                       »
«q_1: ─────────────────────┼─────────────■───────────────────────────────■──»
«     ┌─────────────────┐┌─┴─┐         ┌─┴─┐         ┌────────────────┐┌─┴─┐»
«q_2: ┤ U3(0,0,-3.8456) ├┤ X ├─────────┤ X ├─────────┤ U3(0,0,10.388) ├┤ X ├»
«     └─────────────────┘└───┘         └───┘         └────────────────┘└───┘»
«c_0: ══════════════════════════════════════════════════════════════════════»
«                                                                           »
«c_1: ══════════════════════════════════════════════════════════════════════»
«                                                                           »
«c_2: ══════════════════════════════════════════════════════════════════════»
«                                                                           »
«                                                   ┌──────────────────────┐»
«q_0: ───────────────────■───────────────────────■──┤ U3(-pi/2,-pi/2,pi/2) ├»
«     ┌───────────────┐┌─┴─┐┌─────────────────┐┌─┴─┐└──┬───────────────┬───┘»
«q_1: ┤ U3(pi/2,0,pi) ├┤ X ├┤ U3(0,0,-3.9809) ├┤ X ├───┤ U3(pi/2,0,pi) ├────»
«     ├───────────────┤└───┘└─────────────────┘└───┘   └───────────────┘    »
«q_2: ┤ U3(pi/2,0,pi) ├─────────────────────────────────────────────────────»
«     └───────────────┘                                                     »
«c_0: ══════════════════════════════════════════════════════════════════════»
«                                                                           »
«c_1: ══════════════════════════════════════════════════════════════════════»
«                                                                           »
«c_2: ══════════════════════════════════════════════════════════════════════»
«                                                                           »
«     ┌─────────────────────┐                             »
«q_0: ┤ U3(pi/2,-pi/2,pi/2) ├──■───────────────────────■──»
«     ├─────────────────────┤  │                       │  »
«q_1: ┤ U3(pi/2,-pi/2,pi/2) ├──┼───────────────────────┼──»
«     └─────────────────────┘┌─┴─┐┌─────────────────┐┌─┴─┐»
«q_2: ───────────────────────┤ X ├┤ U3(0,0,-3.8456) ├┤ X ├»
«                            └───┘└─────────────────┘└───┘»
«c_0: ════════════════════════════════════════════════════»
«                                                         »
«c_1: ════════════════════════════════════════════════════»
«                                                         »
«c_2: ════════════════════════════════════════════════════»
«                                                         »
«     ┌──────────────────────┐┌───────────────┐┌───────────┐                  »
«q_0: ┤ U3(-pi/2,-pi/2,pi/2) ├┤ U3(pi/2,0,pi) ├┤ U3(0,0,8) ├──────────────────»
«     └──────────────────────┘└───────────────┘└─